In [1]:
import kagglehub
path = kagglehub.dataset_download("radheshyamkollipara/bank-customer-churn")


import pandas as pd
data = pd.read_csv(path+ "/Customer-Churn-Records.csv")

/home/jorge/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from zorrouno import processor

embedded_data = processor.embbed(data) #esto es literal pd.get_dummies(data)

embedded_data

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Point Earned
0,619,42,2,0.00,1,1,1,101348.88,1,1,2,464
1,608,41,1,83807.86,1,0,1,112542.58,0,1,3,456
2,502,42,8,159660.80,3,1,0,113931.57,1,1,3,377
3,699,39,1,0.00,2,0,0,93826.63,0,0,5,350
4,850,43,2,125510.82,1,1,1,79084.10,0,0,5,425
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,1,300
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,5,771
9997,709,36,7,0.00,1,0,1,42085.58,1,1,3,564
9998,772,42,3,75075.31,2,1,0,92888.52,1,1,2,339


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(embedded_data.drop(["Exited"], axis=1), embedded_data["Exited"], test_size=0.2, random_state=42)
X_train

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Complain,Satisfaction Score,Point Earned
9254,686,32,6,0.00,2,1,1,179093.26,0,2,510
1561,632,42,4,119624.60,2,1,1,195978.86,0,4,959
1670,559,24,3,114739.92,1,1,0,85891.02,1,4,327
6087,561,27,9,135637.00,1,1,0,153080.40,1,2,567
6669,517,56,9,142147.32,1,0,0,39488.04,1,3,727
...,...,...,...,...,...,...,...,...,...,...,...
5734,768,54,8,69712.74,1,1,1,69381.05,0,3,569
5191,682,58,1,0.00,1,1,1,706.50,0,4,709
5390,735,38,1,0.00,3,0,0,92220.12,1,5,678
860,667,43,8,190227.46,1,1,0,97508.04,1,3,680


In [4]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)

EVALUATING. 

Esta parte debería también ir incluida en score.py

In [5]:
import numpy as np

def sigmoid(x):
    return [1 / (1 + np.exp(-y)) for y in x]

y_pred = sigmoid(y_pred)

y_pred_sort = {i:y_pred[i] for i in range(len(y_pred))}
y_pred_sort = dict(sorted(y_pred_sort.items(), key=lambda x: x[1], reverse=True))

how_many_ones = y_test.value_counts()[1]

one_predictions = list(y_pred_sort.keys())[:how_many_ones]

final_preds = [1 if x in one_predictions else 0 for x in range(len(y_pred))]

In [6]:
from sklearn.metrics import classification_report

print(classification_report(y_test, final_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1607
           1       1.00      1.00      1.00       393

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [7]:
umbral = min([y_pred_sort[x] for x in one_predictions])

print(f"CÓMO PREDECIR CON ESTE MODELO:\n\n Paso 1.- Calcularlo con la regresión lineal.\n PASO 2.- Aplicarle sigmoide para acotarlo al intervalo [0,1].\n PASO 3.- Si el resultado es mayor a {umbral} entonces predecir 1.\n          De lo contrario, predecir 0.\n\nAsegúrense de meter esto en score.py")


import json

umb = json.dumps({"umbral": [umbral]})
file = open("umbral.json", "w")
file.write(umb)
file.close()

import pickle

file = open('model.pkl', 'wb')
pickle.dump(reg, file)
file.close()

CÓMO PREDECIR CON ESTE MODELO:

 Paso 1.- Calcularlo con la regresión lineal.
 PASO 2.- Aplicarle sigmoide para acotarlo al intervalo [0,1].
 PASO 3.- Si el resultado es mayor a 0.7285450783333448 entonces predecir 1.
          De lo contrario, predecir 0.

Asegúrense de meter esto en score.py
